In [1]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/Landslide_dataSet3.csv"
#device = "103"
#save_path = f"./data/dev{device}_prepared.csv"
save_path = f"data/dataSet3_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_score', 'iso_risk', 'lstm_score', 'lstm_risk', 'confidence', 'rf_risk', 'devID']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

#def clean_and_format_timestamp(val):
#    if pd.isnull(val):
#        return None
#    val = str(val).replace('(ICT)', '').strip()
#    try:
#        dt = pd.to_datetime(val)
#        return dt.isoformat()
#    except Exception:
#        return None  
#
#df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

#df = df.drop(columns=['Unnamed: 15', 'confidence','rf_risk'], errors='ignore')
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] > 0]
df = df[df['rain'] >= 0]
df = df[df['humi'] > 0]
#df = df[(df['devID'] == 104) | (df['devID'] == 103)]
df = df[(df['geo'] >= 0) & (df['geo'] < 40)]
df = df[df['temp'] > 0]
df = df[(df['devID'] > 0) & (df['devID'] < 1000)]
df = df[df['iso_score'] > -85000]
df = df[df['lstm_score'] >= 0]
#df = df[df['timestamp'] > '3035-09-11']

#df = df[df['devID'] == int(device)]
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
10,2025-11-01 06:42:51.995,108,1.604698,0.0,34.299999,61.299999,1.000000,8.873381,99.764809,0.034925,...,74.550003,15.838068,8.866661,99.754547,0.112785,0.0,0.092680,0.0,0.994999,1.0
11,2025-11-01 06:43:52.076,108,1.604698,0.0,34.500000,60.700001,1.000000,8.872817,99.762062,0.030245,...,64.599998,0.253991,8.886653,99.784546,0.087731,0.0,0.522619,0.0,0.999999,0.0
13,2025-11-01 06:44:52.145,108,1.604698,0.0,34.700001,59.799999,1.000000,8.872817,99.762062,0.023937,...,79.500000,22.393881,8.866809,99.754440,0.089770,0.0,0.031028,0.0,0.995624,1.0
16,2025-11-01 06:45:52.223,108,1.604698,0.0,34.833335,62.600000,1.000000,8.872817,99.762062,0.028857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2025-11-01 06:46:52.303,108,1.604698,0.0,34.900002,62.800001,1.000000,8.872817,99.762062,0.026299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25601,2025-11-07 08:05:17.077,106,39.799999,0.0,33.200001,61.799999,0.451982,8.886656,99.784546,0.082170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25602,2025-11-07 08:05:34.123,105,45.799999,0.0,33.000000,64.900002,0.282659,8.884839,99.783165,0.111972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25603,2025-11-07 08:05:48.770,109,35.900002,0.0,32.099998,65.899998,10.284775,8.868587,99.749573,0.099461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25604,2025-11-07 08:06:02.143,106,39.699999,0.0,33.100000,62.999998,0.411861,8.886655,99.784546,0.080539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
count,24899,24899.00000,24899.000000,24899.000000,24899.000000,24899.000000,24899.000000,24899.000000,24899.000000,24899.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.0,3.000000,3.0,3.000000,3.000000
mean,2025-11-04 16:16:06.916073984,107.57677,35.090856,0.251695,27.051820,85.672530,6.504618,8.872573,99.821136,0.057271,...,72.883334,12.828647,8.873374,99.764511,0.096762,0.0,0.215443,0.0,0.996874,0.666667
min,2025-11-01 06:42:51.995000,105.00000,1.604698,0.000000,21.500000,44.099998,0.000000,1.350000,17.166666,-0.193431,...,64.599998,0.253991,8.866661,99.754440,0.087731,0.0,0.031028,0.0,0.994999,0.000000
25%,2025-11-03 09:48:36.296999936,106.00000,27.000000,0.000000,23.799999,75.699997,0.409782,8.866823,99.749573,0.032107,...,69.575001,8.046030,8.866735,99.754494,0.088751,0.0,0.061854,0.0,0.995312,0.500000
50%,2025-11-04 17:09:38.252999936,108.00000,35.200001,0.000000,25.200001,90.800003,0.961420,8.868587,99.754448,0.054727,...,74.550003,15.838068,8.866809,99.754547,0.089770,0.0,0.092680,0.0,0.995624,1.000000
75%,2025-11-06 00:49:45.191500032,109.00000,41.299999,0.000000,30.600000,96.599998,11.089807,8.886641,99.784538,0.096303,...,77.025002,19.115975,8.876731,99.769547,0.101278,0.0,0.307650,0.0,0.997812,1.000000
max,2025-11-07 08:06:19.199000,109.00000,54.000000,38.277802,38.400002,100.000000,39.624933,8.886712,994.950012,0.183185,...,79.500000,22.393881,8.886653,99.784546,0.112785,0.0,0.522619,0.0,0.999999,1.000000
std,NaN,1.26547,6.497761,2.102902,4.020582,12.854974,8.719325,0.095769,8.091001,0.052177,...,7.588534,11.372605,0.011500,0.017351,0.013914,0.0,0.267803,0.0,0.002724,0.577350


In [3]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_risk', 'iso_score', 'lstm_risk']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
10,2025-11-01 06:42:51.995,108,1.604698,0.0,34.299999,61.299999,1.000000,8.873381,99.764809,0.034925,...,74.550003,15.838068,8.866661,99.754547,0.112785,0.0,0.092680,0.0,0.994999,1.0
11,2025-11-01 06:43:52.076,108,1.604698,0.0,34.500000,60.700001,1.000000,8.872817,99.762062,0.030245,...,64.599998,0.253991,8.886653,99.784546,0.087731,0.0,0.522619,0.0,0.999999,0.0
13,2025-11-01 06:44:52.145,108,1.604698,0.0,34.700001,59.799999,1.000000,8.872817,99.762062,0.023937,...,79.500000,22.393881,8.866809,99.754440,0.089770,0.0,0.031028,0.0,0.995624,1.0
16,2025-11-01 06:45:52.223,108,1.604698,0.0,34.833335,62.600000,1.000000,8.872817,99.762062,0.028857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2025-11-01 06:46:52.303,108,1.604698,0.0,34.900002,62.800001,1.000000,8.872817,99.762062,0.026299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25601,2025-11-07 08:05:17.077,106,39.799999,0.0,33.200001,61.799999,0.451982,8.886656,99.784546,0.082170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25602,2025-11-07 08:05:34.123,105,45.799999,0.0,33.000000,64.900002,0.282659,8.884839,99.783165,0.111972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25603,2025-11-07 08:05:48.770,109,35.900002,0.0,32.099998,65.899998,10.284775,8.868587,99.749573,0.099461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25604,2025-11-07 08:06:02.143,106,39.699999,0.0,33.100000,62.999998,0.411861,8.886655,99.784546,0.080539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
#df['hour'] = df.index.hourผ
#df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24899 entries, 0 to 24898
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    24899 non-null  datetime64[ns]
 1   devID        24899 non-null  int64         
 2   soil         24899 non-null  float64       
 3   rain         24899 non-null  float64       
 4   temp         24899 non-null  float64       
 5   humi         24899 non-null  float64       
 6   geo          24899 non-null  float64       
 7   lat          24899 non-null  float64       
 8   lng          24899 non-null  float64       
 9   iso_score    24899 non-null  float64       
 10  iso_risk     24899 non-null  int64         
 11  lstm_score   24899 non-null  float64       
 12  lstm_risk    24899 non-null  int64         
 13  confidence   24899 non-null  float64       
 14  rf_risk      24896 non-null  float64       
 15  Unnamed: 15  3 non-null      float64       
 16  Unna

In [6]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
rf_risk            3
Unnamed: 15    24896
Unnamed: 16    24896
Unnamed: 17    24896
Unnamed: 18    24896
Unnamed: 19    24896
Unnamed: 20    24896
Unnamed: 21    24896
Unnamed: 22    24896
Unnamed: 23    24896
Unnamed: 24    24896
Unnamed: 25    24896
Unnamed: 26    24896
Unnamed: 27    24896
Unnamed: 28    24896
dtype: int64


In [7]:
#df = df.dropna(subset=['humi']).reset_index(drop=True)

In [8]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
rf_risk            3
Unnamed: 15    24896
Unnamed: 16    24896
Unnamed: 17    24896
Unnamed: 18    24896
Unnamed: 19    24896
Unnamed: 20    24896
Unnamed: 21    24896
Unnamed: 22    24896
Unnamed: 23    24896
Unnamed: 24    24896
Unnamed: 25    24896
Unnamed: 26    24896
Unnamed: 27    24896
Unnamed: 28    24896
dtype: int64


In [9]:
#df = df.dropna()
#
#nan_counts = df.isna().sum()
#nan_columns = nan_counts[nan_counts > 0]
#print("df102")
#print("📌 คอลัมน์ที่มีค่า NaN:")
#print(nan_columns)

In [10]:
#df[(df["iso_risk"]==2) & (df["lstm_risk"]==2)]

In [11]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
count,24899,24899.00000,24899.000000,24899.000000,24899.000000,24899.000000,24899.000000,24899.000000,24899.000000,24899.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.0,3.000000,3.0,3.000000,3.000000
mean,2025-11-04 16:16:06.916073984,107.57677,35.090856,0.251695,27.051820,85.672530,6.504618,8.872573,99.821136,0.057271,...,72.883334,12.828647,8.873374,99.764511,0.096762,0.0,0.215443,0.0,0.996874,0.666667
min,2025-11-01 06:42:51.995000,105.00000,1.604698,0.000000,21.500000,44.099998,0.000000,1.350000,17.166666,-0.193431,...,64.599998,0.253991,8.866661,99.754440,0.087731,0.0,0.031028,0.0,0.994999,0.000000
25%,2025-11-03 09:48:36.296999936,106.00000,27.000000,0.000000,23.799999,75.699997,0.409782,8.866823,99.749573,0.032107,...,69.575001,8.046030,8.866735,99.754494,0.088751,0.0,0.061854,0.0,0.995312,0.500000
50%,2025-11-04 17:09:38.252999936,108.00000,35.200001,0.000000,25.200001,90.800003,0.961420,8.868587,99.754448,0.054727,...,74.550003,15.838068,8.866809,99.754547,0.089770,0.0,0.092680,0.0,0.995624,1.000000
75%,2025-11-06 00:49:45.191500032,109.00000,41.299999,0.000000,30.600000,96.599998,11.089807,8.886641,99.784538,0.096303,...,77.025002,19.115975,8.876731,99.769547,0.101278,0.0,0.307650,0.0,0.997812,1.000000
max,2025-11-07 08:06:19.199000,109.00000,54.000000,38.277802,38.400002,100.000000,39.624933,8.886712,994.950012,0.183185,...,79.500000,22.393881,8.886653,99.784546,0.112785,0.0,0.522619,0.0,0.999999,1.000000
std,NaN,1.26547,6.497761,2.102902,4.020582,12.854974,8.719325,0.095769,8.091001,0.052177,...,7.588534,11.372605,0.011500,0.017351,0.013914,0.0,0.267803,0.0,0.002724,0.577350


In [12]:
df['devID'].value_counts()

devID
109    8374
106    7794
108    5893
107    2595
105     243
Name: count, dtype: int64

In [13]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
